# Projeto 2 - Classificador Automático de Sentimento

Você foi contratado por uma empresa parar analisar como os clientes estão reagindo a um determinado produto no Twitter. A empresa deseja que você crie um programa que irá analisar as mensagens disponíveis e classificará como "relevante" ou "irrelevante". Com isso ela deseja que mensagens negativas, que denigrem o nome do produto, ou que mereçam destaque, disparem um foco de atenção da área de marketing.<br /><br />
Como aluno de Ciência dos Dados, você lembrou do Teorema de Bayes, mais especificamente do Classificador Naive-Bayes, que é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser relevante dadas as palavras em seu conteúdo.<br /><br />
Para realizar o MVP (*minimum viable product*) do projeto, você precisa implementar uma versão do classificador que "aprende" o que é relevante com uma base de treinamento e compara a performance dos resultados com uma base de testes.<br /><br />
Após validado, o seu protótipo poderá também capturar e classificar automaticamente as mensagens da plataforma.

## Informações do Projeto

Prazo: 13/Set até às 23:59.<br />
Grupo: 1 ou 2 pessoas.<br /><br />
Entregáveis via GitHub: 
* Arquivo notebook com o código do classificador, seguindo as orientações abaixo.
* Arquivo Excel com as bases de treinamento e teste totalmente classificado.

**NÃO disponibilizar o arquivo com os *access keys/tokens* do Twitter.**


### Check 3: 

Até o dia 06 de Setembro às 23:59, o notebook e o xlsx devem estar no Github com as seguintes evidências: 
    * Conta no twitter criada.
    * Produto escolhido.
    * Arquivo Excel contendo a base de treinamento e teste já classificado.

Sugestão de leitura:<br />
http://docs.tweepy.org/en/v3.5.0/index.html<br />
https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/

___

## Preparando o ambiente

Instalando a biblioteca *tweepy* para realizar a conexão com o Twitter:

In [1]:
%%capture

#Instalando o tweepy
!pip install tweepy

Importando as Bibliotecas que serão utilizadas. Esteja livre para adicionar outras.

In [1]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle
import numpy as np

___
## Autenticando no  Twitter

Para realizar a captura dos dados é necessário ter uma conta cadastrada no twitter:

* Conta: ***@Cd2aProjeto***


1. Caso ainda não tenha uma: https://twitter.com/signup
1. Depois é necessário registrar um app para usar a biblioteca: https://apps.twitter.com/
1. Dentro do registro do App, na aba Keys and Access Tokens, anotar os seguintes campos:
    1. Consumer Key (API Key)
    1. Consumer Secret (API Secret)
1. Mais abaixo, gere um Token e anote também:
    1. Access Token
    1. Access Token Secret
    
1. Preencha os valores no arquivo "auth.pass"

**ATENÇÃO**: Nunca divulgue os dados desse arquivo online (GitHub, etc). Ele contém as chaves necessárias para realizar as operações no twitter de forma automática e portanto é equivalente a ser "hackeado". De posse desses dados, pessoas mal intencionadas podem fazer todas as operações manuais (tweetar, seguir, bloquear/desbloquear, listar os seguidores, etc). Para efeito do projeto, esse arquivo não precisa ser entregue!!!

In [ ]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @Cd2aProjeto

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Coletando Dados

Agora vamos coletar os dados. Tenha em mente que dependendo do produto escolhido, não haverá uma quantidade significativa de mensagens, ou ainda poder haver muitos retweets.<br /><br /> 
Configurando:

In [ ]:
#Produto escolhido:
produto = 'nubank'

#Quantidade mínima de mensagens capturadas:
n = 500
#Quantidade mínima de mensagens para a base de treinamento:
t = 300

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

Capturando os dados do twitter:

In [ ]:
#Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang).items():    
    msgs.append(msg.text.lower())
    i += 1
    if i > n:
        break

#Embaralhando as mensagens para reduzir um possível viés
shuffle(msgs)

Salvando os dados em uma planilha Excel:

In [ ]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()

___
## Classificando as Mensagens

Agora você deve abrir o arquivo Excel com as mensagens capturadas e classificar na Coluna B se a mensagem é relevante ou não.<br /> 
Não se esqueça de colocar um nome para a coluna na célula **B1**.<br /><br />
Fazer o mesmo na planilha de Controle.

___
## Montando o Classificador Naive-Bayes

Com a base de treinamento montada, comece a desenvolver o classificador. Escreva o seu código abaixo:

Opcionalmente: 
* Limpar as mensagens removendo os caracteres: enter, :, ", ', (, ), etc. Não remover emojis.<br />
* Corrigir separação de espaços entre palavras e/ou emojis.
* Propor outras limpezas/transformações que não afetem a qualidade da informação.



In [2]:
nubank = pd.read_excel('nubank.xlsx', sep = ',')


nubank.Treinamento = nubank.Treinamento.replace('_','',regex=True)
nubank.Treinamento = nubank.Treinamento.replace(',','',regex=True)
nubank.Treinamento = nubank.Treinamento.replace('@','',regex=True)
nubank.Treinamento = nubank.Treinamento.replace('#','',regex=True)
nubank.Treinamento = nubank.Treinamento.replace('  ','',regex=True)
nubank.Treinamento = nubank.Treinamento.replace(':','',regex=True)
nubank.Treinamento = nubank.Treinamento.replace('"','',regex=True)
nubank.Treinamento = nubank.Treinamento.replace('rt','',regex=True)
nubank.Treinamento = nubank.Treinamento.replace('\\*','',regex=True)
nubank.Treinamento = nubank.Treinamento.replace('\*','',regex=True)
nubank.Treinamento = nubank.Treinamento.replace('\$','',regex=True)
nubank.Treinamento = nubank.Treinamento.replace('\n','',regex=True)
nubank.Treinamento = nubank.Treinamento.replace('//','',regex=True)
nubank.Treinamento = nubank.Treinamento.replace(';','',regex=True)
nubank.Treinamento = nubank.Treinamento.replace('/','',regex=True)
nubank.Treinamento = nubank.Treinamento.replace('\(','',regex=True)
nubank.Treinamento = nubank.Treinamento.replace('\)','',regex=True)
nubank.Treinamento = nubank.Treinamento.replace('\'','',regex=True)
nubank.Treinamento = nubank.Treinamento.replace('!','',regex=True)

In [3]:
#Separando colunas de Treinamento e Relevancia
treina = pd.Series(np.sum(nubank.Treinamento + '').split())
relev = pd.Series(np.sum(nubank.Relevancia + '').split())

In [4]:
#Separando tabelas de Treinamento e Relevancia
data_relev = nubank[nubank.Relevancia == 'r']
data_irrelev = nubank[nubank.Relevancia == 'ir']

#Soma da tabela dos relevantas
soma_relev = pd.Series(np.sum(data_relev.Treinamento + '').split())

#Lista dos relevantes
lista_relev = np.sum(data_relev.Treinamento+'').split()

#Calculo do total dos relevantes
total_relev = len(lista_relev)

#Calculo do total dos irrelevantes
soma_irrelev = pd.Series(np.sum(data_irrelev.Treinamento+'').split())

#Lista dos irrelevantes
lista_irrelev = np.sum(data_irrelev.Treinamento+'').split()

#Calculo do total de irrelevantes
total_irrelev = len(lista_irrelev)

In [5]:
#Função para calcular probabilidade de ser R ou I
def bayes(lista):
    lista = lista.split(' ')
    prob_R = []
    prob_I = []
    for i in range(len(lista)):
        prob_R.append((lista_relev.count(lista[i])+1)/(total_relev+len(lista)))
        prob_I.append((lista_irrelev.count(lista[i])+1)/(total_irrelev+len(lista)))
    
    #Multiplica todos os fatores da lista
    R = np.prod(prob_R)
    I = np.prod(prob_I)
    
    if R>I:
        return 'r'
    else:
        return 'ir'

In [6]:
#Testando função
tweet_R = "o odio q eu to do itaú....... eh agora q eu mudo pro nubank viu"
tweet_I = "rt @nubankbrasil: diga olá para uma vida livre de dificuldades. peça seu cartão: https://t.co/n6igyi01pm https://t.co/a45pu21eov"

print(bayes(tweet_I) + ' <---  X  ---> ' +bayes(tweet_R))

ir <---  X  ---> r


___
## Verificando a performance

Agora você deve testar o seu Classificador com a base de Testes.<br /><br /> 

Você deve extrair as seguintes medidas:
* Porcentagem de positivos falsos (marcados como relevante mas não são relevantes)
* Porcentagem de positivos verdadeiros (marcado como relevante e são relevantes)
* Porcentagem de negativos verdadeiros (marcado como não relevante e não são relevantes)
* Porcentagem de negativos falsos (marcado como não relevante e são relevantes)

Opcionalmente:
* Criar categorias intermediárias de relevância baseado na diferença de probabilidades. Exemplo: muito relevante, relevante, neutro, irrelevante e muito irrelevante.

In [9]:
nubank = pd.read_excel('nubank.xlsx', sheetname = 'Teste', sep = ',')
nubank['Maquina'] = ''

nubank.Teste = nubank.Teste.replace('_','',regex=True)
nubank.Teste = nubank.Teste.replace(',','',regex=True)
nubank.Teste = nubank.Teste.replace('@','',regex=True)
nubank.Teste = nubank.Teste.replace('#','',regex=True)
nubank.Teste = nubank.Teste.replace('  ','',regex=True)
nubank.Teste = nubank.Teste.replace(':','',regex=True)
nubank.Teste = nubank.Teste.replace('"','',regex=True)
nubank.Teste = nubank.Teste.replace('rt','',regex=True)
nubank.Teste = nubank.Teste.replace('\\*','',regex=True)
nubank.Teste = nubank.Teste.replace('\*','',regex=True)
nubank.Teste = nubank.Teste.replace('\$','',regex=True)
nubank.Teste = nubank.Teste.replace('\n','',regex=True)
nubank.Teste = nubank.Teste.replace('//','',regex=True)
nubank.Teste = nubank.Teste.replace(';','',regex=True)
nubank.Teste = nubank.Teste.replace('/','',regex=True)
nubank.Teste = nubank.Teste.replace('\(','',regex=True)
nubank.Teste = nubank.Teste.replace('\)','',regex=True)
nubank.Teste = nubank.Teste.replace('\'','',regex=True)
nubank.Teste = nubank.Teste.replace('!','',regex=True)


In [10]:
#Criando a lista relevantes
relevantes = []
for i in range(len(nubank.Teste)):
    relevantes.append(bayes(nubank.Teste[i]))
    

#Adicionando a coluna Maquina do dataFrame
nubank.Maquina = pd.Series(relevantes)
nubank.head()

,Teste,Relevancia,Maquina
0,oi bbsalguém me envia um convite do nubankobri...,r,r
1,nubankbrasil diga olá para uma vida livre de ...,ir,ir
2,eliezer94589997 vc pode pedir através de nosso...,r,r
3,nubankbrasil diga olá para uma vida livre de ...,ir,ir
4,nubankbrasil unimos design e tecnologia para ...,ir,ir


In [11]:
#Analisando a performance
p = []
for i in range(len(nubank.Teste)):
    if nubank.Relevancia[i] == 'r':
        if nubank.Maquina[i] == 'r':
            p.append('PV')
        if nubank.Maquina[i] == 'ir':
            p.append('NF')
    elif nubank.Relevancia[i] == 'ir':
        if nubank.Maquina[i] == 'ir':
            p.append('NV')
        if nubank.Maquina[i] == 'r':
            p.append('PF')
#Criando serie de acertos
acertos = pd.Series(p)


In [12]:
pf = p.count('PF')/2
pv = p.count('PV')/2
nf = p.count('NF')/2
nv = p.count('NV')/2

print("Positivos Falsos: {0}%\nPositivos Verdadeiros: {1}%\nNegativos Verdadeiros: {2}%\nNegativos Falsos: {3}%".format(pf,pv,nv,nf))

Positivos Falsos: 2.5%
Positivos Verdadeiros: 53.0%
Negativos Verdadeiros: 40.5%
Negativos Falsos: 4.0%


___
## Conclusão

Ao iniciarmos o projeto, houve duvida sobre proceder com o uso da biblioteca Pandas ou com listas, portanto fizemos dos dois jeitos (existem dois arquivos jupyter notebook no git). Decidimos usar esse pois ao compararmos os Positivos e Negativos Verdadeiros, conseguimos chegar a um resultado mais preciso usando a biblioteca.
Como podemos ver nos resultados acima, temos uma precisão de 93,5% de acertos na classificação de tweets, mas mesmo assim encontramos dois problemas principais para realizar o Classificador pela Teoria de Bayes:

* Mensagens sarcasticas ou com dupla negação, que são tratadas como qualquer outra, fazendo com que não se encaixem na teoria, já que esta nao reconhece subjetividade nas mensagens. Ao digitar duas vezes a palavra "não", por exemplo, o sistema simplesmente reconhece as duas palavras independentemente de sua correlação.
* Definir o tamanho dos dados utilizados para treinamento, pois como o projeto foi apenas um teste, não pudemos expandir a base de dados inicial para fazer o Classificador. Quanto maior for a base de dados inicial, maior será a precisão que teremos no resultado final, pois teremos mais informações sobre a interação das palavras.

Se aprovado, o projeto já tem um plano de expansão definido, que consistiria em expandir a base de Treinamento e fazer com que o programa passe a reconhecer dentro de tweets elementos como dupla negação e outros que tornariam a classificação dos tweets mais precisa e personalizada para o desenvolvimento do produto.
